In [1]:
import transformers
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-cased"  # cased means Upper and Lower case are distinguished
tokenizer = AutoTokenizer.from_pretrained(model_name)




tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [2]:
# Input text
text = 'Hello, welcome to SMU MITB Gen AI and LLM Course. What is your name?'


In [3]:
tokens = tokenizer(text)

In [4]:
print(tokens['input_ids'])

[101, 8667, 117, 7236, 1106, 19293, 2591, 13219, 2064, 9198, 19016, 1105, 12427, 2107, 10176, 119, 1327, 1110, 1240, 1271, 136, 102]


In [5]:
print(tokens)

{'input_ids': [101, 8667, 117, 7236, 1106, 19293, 2591, 13219, 2064, 9198, 19016, 1105, 12427, 2107, 10176, 119, 1327, 1110, 1240, 1271, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
# Tokenize and encode the text
tokens2 = tokenizer.encode(text)

In [7]:
# Print the token IDs
print( tokens2)

[101, 8667, 117, 7236, 1106, 19293, 2591, 13219, 2064, 9198, 19016, 1105, 12427, 2107, 10176, 119, 1327, 1110, 1240, 1271, 136, 102]


In [8]:
# Convert token IDs back to tokens
tokens3 = tokenizer.convert_ids_to_tokens(tokens['input_ids'])

# Print the corresponding tokens
print("Tokens:", tokens3)

Tokens: ['[CLS]', 'Hello', ',', 'welcome', 'to', 'SM', '##U', 'MIT', '##B', 'Gen', 'AI', 'and', 'LL', '##M', 'Course', '.', 'What', 'is', 'your', 'name', '?', '[SEP]']


In [9]:
tokens4= tokenizer.convert_ids_to_tokens(tokens2)
#print the corresponding tokens
print("Tokens:", tokens4)

Tokens: ['[CLS]', 'Hello', ',', 'welcome', 'to', 'SM', '##U', 'MIT', '##B', 'Gen', 'AI', 'and', 'LL', '##M', 'Course', '.', 'What', 'is', 'your', 'name', '?', '[SEP]']


In [10]:
model = transformers.BertModel.from_pretrained("bert-base-cased")


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [11]:
 embedding_layer = model.embeddings

In [12]:
### Access the word embeddings within the BertEmbeddings layer
### as opposed to the entire embedding layer that also has
### position embeddings and token type embeddings
word_embeddings = embedding_layer.word_embeddings

# Now view the embedding values
print(word_embeddings.weight)

Parameter containing:
tensor([[-0.0005, -0.0416,  0.0131,  ..., -0.0039, -0.0335,  0.0150],
        [ 0.0169, -0.0311,  0.0042,  ..., -0.0147, -0.0356, -0.0036],
        [-0.0006, -0.0267,  0.0080,  ..., -0.0100, -0.0331, -0.0165],
        ...,
        [-0.0064,  0.0166, -0.0204,  ..., -0.0418, -0.0492,  0.0042],
        [-0.0048, -0.0027, -0.0290,  ..., -0.0512,  0.0045, -0.0118],
        [ 0.0313, -0.0297, -0.0230,  ..., -0.0145, -0.0525,  0.0284]],
       requires_grad=True)


In [13]:
word_embeddings.weight.shape

torch.Size([28996, 768])

Lets make use of the BERT embeddings of single words now

In [14]:
word1 = 'dog'
print(tokenizer(word1))


{'input_ids': [101, 3676, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [15]:
word2='puppy'
print(tokenizer(word2))

{'input_ids': [101, 21566, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


Now, let's compute cosine similarity of the words

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

remove CLS and SEP tokens for this computation

In [17]:
token_ids1 = tokenizer.encode(word1,  add_special_tokens=False)
token_ids2 = tokenizer.encode(word2,  add_special_tokens=False)

In [18]:
# Re-select embeddings from embeddings layer without the extra tokens this time
embedding1 = embedding_layer.word_embeddings.weight[token_ids1]
embedding2 = embedding_layer.word_embeddings.weight[token_ids2]
# Convert tensors to NumPy arrays and squeeze result to remove extra dimensions
embedding1 = embedding1.squeeze().detach().numpy()
embedding2 = embedding2.squeeze().detach().numpy()

In [19]:
# Norms of the embeddings
magnitude1 = np.linalg.norm(embedding1)
magnitude2 = np.linalg.norm(embedding2)

# Cosine similarity
cosine_sim = np.dot(embedding1, embedding2) / (magnitude1 * magnitude2)

In [20]:
print(f"Cosine sim of '{word1}' and '{word2}': {cosine_sim}")

Cosine sim of 'dog' and 'puppy': 0.44258785247802734


Let's compare with using sklearn to compute the cosine sim

In [21]:
cosine_sim = cosine_similarity([embedding1], [embedding2])

print(f"Cosine sim of '{word1}' and '{word2}': {cosine_sim}")

Cosine sim of 'dog' and 'puppy': [[0.44258782]]


We can see that results are the same

Now, let's redo the computation but leveraging the model through a forward pass to take into account context

In [22]:
import torch

model = AutoModel.from_pretrained(model_name)

# Now we need the full output of the tokeniser for the forward pass, not just encode

token_ids1 = tokenizer(word1, return_tensors="pt",  add_special_tokens=False)
token_ids2 = tokenizer(word2, return_tensors="pt",  add_special_tokens=False)


## Get embeddings from BERT model forward pass, do not store values needed for gradient comp (no_grad)
# Now use token_ids1 and 2 since we need all compunents of the embeddings

with torch.no_grad():
    # Forward pass to get embeddings from the last hidden state
    outputs1 = model(**token_ids1)
    outputs2 = model(**token_ids2)

# Extract embeddings for first token (word embedding)
# The last_hidden_state is of shape (batch_size, sequence_length, hidden_dim)
# Use average pooling over the sequence dimension (dim=1) to get a fixed-size embedding
embedding1 = outputs1.last_hidden_state.mean(dim=1).squeeze().numpy()
embedding2 = outputs2.last_hidden_state.mean(dim=1).squeeze().numpy()

# Now let's take their cosine sim as before

cosine_sim = cosine_similarity([embedding1], [embedding2])

print(f"Cosine sim of '{word1}' and '{word2}': {cosine_sim}")



Cosine sim of 'dog' and 'puppy': [[0.7207791]]


In [23]:
token_ids1 = tokenizer(word1, return_tensors="pt",  add_special_tokens=True)
token_ids2 = tokenizer(word2, return_tensors="pt",  add_special_tokens=True)


## Get embeddings from BERT model forward pass, do not store values
## needed for gradient comp (no_grad)
# Now use token_ids1 and 2 since we need all compunents of the embeddings

with torch.no_grad():
    # Forward pass to get embeddings from the last hidden state
    outputs1 = model(**token_ids1)
    outputs2 = model(**token_ids2)

# Extract embeddings for first token (word embedding)
# The last_hidden_state is of shape (batch_size, sequence_length, hidden_dim)
# Use average pooling over the sequence dimension (dim=1) to get a fixed-size embedding
embedding1 = outputs1.last_hidden_state.mean(dim=1).squeeze().numpy()
embedding2 = outputs2.last_hidden_state.mean(dim=1).squeeze().numpy()

# Now let's take their cosine sim as before

cosine_sim = cosine_similarity([embedding1], [embedding2])

print(f"Cosine sim of '{word1}' and '{word2}': {cosine_sim}")

Cosine sim of 'dog' and 'puppy': [[0.9237887]]


Let's try comparing the similarity of the CLS tokens of each word instead

In [24]:
token_ids1 = tokenizer(word1, return_tensors="pt",  add_special_tokens=True)
token_ids2 = tokenizer(word2, return_tensors="pt")  #default is to return all tokens including special tokens

with torch.no_grad():
    # Forward pass to get embeddings from the last hidden state
    outputs1 = model(**token_ids1)
    outputs2 = model(**token_ids2)

# Extract embeddings for  [CLS] token (index 0 of last_hidden_state)
# The `last_hidden_state` has shape (batch_size, sequence_length, hidden_dim)
embedding1 = outputs1.last_hidden_state[0, 0].numpy()  # [CLS] token for word1
embedding2 = outputs2.last_hidden_state[0, 0].numpy()  # [CLS] token for word2

# Now let's take their cosine sim as before

cosine_sim = cosine_similarity([embedding1], [embedding2])

print(f"Cosine sim of '{word1}' and '{word2}': {cosine_sim}")

Cosine sim of 'dog' and 'puppy': [[0.93784463]]


------------------------
**Let's use more than just words now and see the benefit of the attention mechanism**
------------------------

In [25]:
import torch, transformers
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-cased"  # cased means Upper and Lower case are distinguished
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = transformers.BertModel.from_pretrained("bert-base-cased")
embedding_layer = model.embeddings

In [26]:
sent1 = "My favourite subject at university is Gen AI."
sent2 = "You will soon be experts in artificial intelligence!"

In [27]:
tokens_sent1 = tokenizer.encode(sent1)
tokens_sent2 = tokenizer.encode(sent2)

Using tokenized sentences, retrieve BERT STATIC word embeddings (from embedding layer of pre-trained model), average them (mean-pooling) to get sentence level embeddings.  Convert into NumPy arrays before computing cosine sim with numpy

In [28]:
s_embedding1 = embedding_layer.word_embeddings.weight[tokens_sent1].mean(dim=0).squeeze().detach().numpy()
s_embedding2 = embedding_layer.word_embeddings.weight[tokens_sent2].mean(dim=0).squeeze().detach().numpy()

In [29]:
cosine_sim_static = cosine_similarity([s_embedding1], [s_embedding2])

print(f"Cosine sim of \n'{sent1}'\n and '{sent2}'\n is:  {cosine_sim_static}")

Cosine sim of 
'My favourite subject at university is Gen AI.'
 and 'You will soon be experts in artificial intelligence!'
 is:  [[0.5120559]]


Now let's again do a forward pass on the 2 sentences and compute sentence level sim in 2 different ways: (1) by mean pooling and (2) by CLS comparison

In [30]:
sent_ids1 = tokenizer(sent1, return_tensors="pt", padding=True)
## pad so that 2 sentences same length in tokens -
#  as we will be passing them through the model as a BATCH]
sent_ids2 = tokenizer(sent2, return_tensors="pt", padding=True)
## Get embeddings from BERT model forward pass, do not store values needed for gradient comp (no_grad)
with torch.no_grad():
    # Forward pass to get embeddings from the last hidden state
    outputs1 = model(**sent_ids1)
    outputs2 = model(**sent_ids2)

s_embedding3 = outputs1.last_hidden_state.mean(dim=1).squeeze().numpy()
s_embedding4 = outputs2.last_hidden_state.mean(dim=1).squeeze().numpy()

cosine_sim_mean = cosine_similarity([s_embedding3], [s_embedding4])

print(f"Cosine sim with BERT forward is {cosine_sim_mean} \n"
      f"With STATIC embeddings it was {cosine_sim_static}")

Cosine sim with BERT forward is [[0.8084202]] 
With STATIC embeddings it was [[0.5120559]]


Same thing but compare ]CLS] token similarity instead of average pooling of all tokens embeddings

In [31]:
# Extract embeddings for  [CLS] token (index 0 of last_hidden_state)
# The `last_hidden_state` has shape (batch_size, sequence_length, hidden_dim)
embedding5 = outputs1.last_hidden_state[0, 0].numpy()  # [CLS] token for sent1
embedding6 = outputs2.last_hidden_state[0, 0].numpy()

# Now let's take their cosine sim as before

cosine_sim_CLS = cosine_similarity([embedding5], [embedding6])

print(f"Cosine sim of [CLS] tokens is  {cosine_sim_CLS} \n"
f"Cosine sim with mean pooling is {cosine_sim_mean} \n"
f"and STATIC embeddings was {cosine_sim_static}"
)

Cosine sim of [CLS] tokens is  [[0.9434209]] 
Cosine sim with mean pooling is [[0.8084202]] 
and STATIC embeddings was [[0.5120559]]
